In [1]:
import pandas as pd
import numpy as np


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm, tqdm_notebook, trange
from scipy.optimize import fmin
from pylab import rcParams
from IPython.display import clear_output
import copy
rcParams['figure.figsize'] = 8, 5

C:\Users\shankar\Anaconda3\envs\tf\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [152]:
# Imports for better visualization
from matplotlib import rcParams
#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 4)
rcParams['figure.dpi'] = 150
#rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['font.size'] = 8
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'
rcParams['axes.grid'] = True
rcParams['axes.facecolor'] = '#eeeeee'

pd.set_option('display.max_columns', None)

In [153]:
train = pd.read_csv('train_1081_feat.csv')
test = pd.read_csv('test_1081_feat.csv')
sample = pd.read_excel('Sample_Submission.xlsx')
train.shape, test.shape, sample.shape

((947, 1082), (527, 1081), (527, 1))

In [148]:
drop_cols = ['Freq_Of_Word_40']

In [155]:
X = train.drop(['IsGoodNews']+drop_cols, axis=1)
y = train['IsGoodNews']
test_X = test.drop(drop_cols, axis=1)

X.shape, test_X.shape

((947, 1080), (527, 1080))

In [156]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
import copy
from sklearn.metrics import accuracy_score

In [160]:
m

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [167]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier

from scipy.optimize import fmin 

err_score = []
test_pred_lgb = []

for i in range(10):
    
    err_lgb_ac = []
    err_lgb_f1 = []
    err_lgb_f1_c = []
    err_lgb_auc = []

    y_pred_tot_lgb = []
    y_pred_tot_lgb_c = []
    y_pred_tot_lgb_prob = []

    cutoff_value = []
    feature_importance_df = pd.DataFrame()

    fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=int(i*24))
    i = 1
    f = 0
    for train_index, test_index in fold.split(X, y):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        m=LGBMClassifier(random_state=int(i*12), scale_pos_weight=2)

        m.fit(X_train,y_train)

        y_pred = m.predict(X_test)
        y_pred_prob = m.predict_proba(X_test)[:,1]

        test_pred = m.predict(test_X)
        test_pred_proba = m.predict_proba(test_X)[:,1]


        err_ac = accuracy_score(y_test, y_pred)
        err_f1 = f1_score(y_test, y_pred)
        err_auc = roc_auc_score(y_test, y_pred_prob)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = X_train.columns
        fold_importance_df["importance"] = m.feature_importances_
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)


        f += 1
        print("Fold:", f)
        print("Accuracy:", err_ac)
        print('F1 Score:', err_f1)
        print("AUC Score:", err_auc)
        print('\n')

        err_lgb_ac.append(err_ac)
        err_lgb_f1.append(err_f1)
        err_lgb_auc.append(err_auc)

        y_pred_tot_lgb.append(test_pred)
        y_pred_tot_lgb_prob.append(test_pred_proba)
    
    err_score.append(np.mean(err_lgb_f1))
    test_pred_lgb.append(np.mean(y_pred_tot_lgb, 0).round().astype(int))
    
    print("Fold Score:", np.mean(err_lgb_f1))
    print()
    

Fold: 1
Accuracy: 0.9210526315789473
F1 Score: 0.9006622516556292
AUC Score: 0.97856477166822


Fold: 2
Accuracy: 0.9368421052631579
F1 Score: 0.923076923076923
AUC Score: 0.9836323392357875


Fold: 3
Accuracy: 0.9523809523809523
F1 Score: 0.9379310344827587
AUC Score: 0.98930669800235


Fold: 4
Accuracy: 0.91005291005291
F1 Score: 0.881118881118881
AUC Score: 0.9769721303731695


Fold: 5
Accuracy: 0.9417989417989417
F1 Score: 0.9271523178807948
AUC Score: 0.985710911667454


Fold Score: 0.9139882816429974

Fold: 1
Accuracy: 0.9052631578947369
F1 Score: 0.8860759493670887
AUC Score: 0.97856477166822


Fold: 2
Accuracy: 0.9157894736842105
F1 Score: 0.8904109589041096
AUC Score: 0.9762348555452004


Fold: 3
Accuracy: 0.9153439153439153
F1 Score: 0.8974358974358975
AUC Score: 0.9800235017626322


Fold: 4
Accuracy: 0.9259259259259259
F1 Score: 0.9027777777777778
AUC Score: 0.9731931979215871


Fold: 5
Accuracy: 0.9470899470899471
F1 Score: 0.9315068493150684
AUC Score: 0.990198393953708




In [168]:
err_score

[0.9139882816429974,
 0.9016414865599884,
 0.9002641957590816,
 0.9074118390836657,
 0.9121797589670615,
 0.9089961900999952,
 0.8998288141142602,
 0.9047374737562123,
 0.9128177400316544,
 0.9063781894396788]

In [169]:
np.mean(err_score)

0.9068243969454596

In [173]:
lgb_test_pred1 = np.mean(test_pred_lgb, 0).round().astype(int)

sample['IsGoodNews'] = lgb_test_pred1
print(sample['IsGoodNews'].value_counts())
sample.to_excel('Models_10_EXP_1.xlsx', index=False) #

0    325
1    202
Name: IsGoodNews, dtype: int64
